In [1]:
from wandb.sdk import wandb_config

import wandb
import torch
from model.train import train
from utils.configs import *
from data.CommentaryDataloader import get_commentary_dataloader
from data.CommentaryDataset import CommentaryDataset
from model.predict import Predictor
import wandb

In [8]:
def get_configs_from_wandb_config(wandb_config: WandbConfig, is_real_wandb: bool):
    if wandb_config['model_name'] not in ['model', 'model_residual_encoder']:
        raise ValueError('Model should be model or model_residual_encoder')
    
    shared_config: SharedConfig = {
        'context_length': wandb_config['context_length'],
        'sentencepiece_path': f"./artifacts/sp{wandb_config['sp_vocab']}.model",
        'bos_id': 0, # will be initialized later
        'eos_id': 1, # will be initialized later
        'pad_id': 2, # will be initialized later
        'vocab_size': 0, # will be initialized later
        'target_types': wandb_config['target_types']
    }
    
    train_data_config: DataConfig = {
        'batch_size': wandb_config['batch_size'],
        'split': 'train',
        'data_path': './processed_data',
        'past_boards': wandb_config['past_boards'],
        'stride_big_sequences': wandb_config['stride_big_sequences'],
        'in_memory': False,
        'dl_shuffle': False,
        'dl_samples': wandb_config['samples_per_train_epoch'],
        'dl_num_workers': 2,
    }
    
    valid_data_config: DataConfig = {
        'batch_size': wandb_config['batch_size'],
        'split': 'valid',
        'data_path': './processed_data',
        'past_boards': wandb_config['past_boards'],
        'stride_big_sequences': wandb_config['stride_big_sequences'],
        'in_memory': True,
        'dl_shuffle': True,
        'dl_samples': None,
        'dl_num_workers': 1,
    }
    
    test_data_config: DataConfig = {
        'batch_size': wandb_config['batch_size'],
        'split': 'valid',
        'data_path': './processed_data',
        'past_boards': wandb_config['past_boards'],
        'stride_big_sequences': wandb_config['stride_big_sequences'],
        'in_memory': False,
        'dl_shuffle': False,
        'dl_samples': None,
        'dl_num_workers': 0,
    }
    
    model_config: ModelConfig = {
        'name': Models.MODEL_RESIDUAL_ENCODER if wandb_config['model_name'] == 'model_residual_encoder' else Models.MODEL,
        'board_embedding_size': wandb_config['board_embedding_size'],
        'text_embedding_size': wandb_config['text_embedding_size'],
        'conv_modules_count': wandb_config['conv_modules_count'],
        'transformer_blocks': wandb_config['transformer_blocks'],
        'board_intermediary_channels': wandb_config['board_intermediary_channels'],
        'board_in_channels': CommentaryDataset.get_board_channels(train_data_config),
        'board_height': 8,
        'board_width': 8,
        'board_depth': wandb_config['board_embedding_size'],
        'ff_inner_channels': wandb_config['ff_inner_channels'],
        'num_heads': wandb_config['num_heads']
    }
    
    if wandb_config['optimizer'] not in ['adam', 'sgd']:
        raise ValueError('Optimizer value invalid')
    
    train_config: TrainConfig = {
        'lr': wandb_config['lr'],
        'with_wandb': is_real_wandb,
        'num_epochs': wandb_config['num_epochs'],
        'predict_sentences': wandb_config['predict_sentences'],
        'optimizer': Optimizers.ADAM if wandb_config['optimizer'] == 'adam' else Optimizers.SGD
    }
    
    return {
        'shared_config': shared_config,
        'train_config': train_config,
        'model_config': model_config,
        'test_data_config': test_data_config,
        'train_data_config': train_data_config,
        'valid_data_config': valid_data_config
    }     

In [9]:
# local_wandb_config: WandbConfig = {
#     'model_name': 'model',
#     'text_embedding_size': 64,
#     'conv_modules_count': 6,
#     'transformer_blocks': 6,
#     'board_intermediary_channels': 64,
#     'board_embedding_size': 64,
#     'ff_inner_channels': 64,
#     'num_heads': 4,
#     'lr': 0.01,
#     'optimizer': 'sgd',
#     'num_epochs': 200,
#     'context_length': 256,
#     'sp_vocab': 800,
#     'batch_size': 64,
#     'past_boards': 1,
#     'stride_big_sequences': 64,
#     'samples_per_train_epoch': 100000,
#     'predict_sentences': 4,
#     'target_types': [0]
# }

sweep_config = {
    "method": "random",
    "metric": {"goal": "minimize", "name": "val_loss"},
    "parameters": {
        'model_name': {"values": ['model_residual_encoder', 'model'] },
        'text_embedding_size': {"distribution": "q_log_uniform_values", "min": 64, "max": 256, "q": 64},
        'conv_modules_count': {"values": [1, 2, 3, 4]},
        'transformer_blocks': {"values": [1, 2, 3, 4]},
        'board_intermediary_channels': {"distribution": "q_log_uniform_values", "min": 64, "max": 512, "q": 64},
        'board_embedding_size': {"distribution": "q_log_uniform_values", "min": 64, "max": 256, "q": 64},
        'ff_inner_channels': {"distribution": "q_log_uniform_values", "min": 64, "max": 512, "q": 64},
        'num_heads': {"distribution": "q_log_uniform_values", "min": 4, "max": 8, "q": 4},
        'lr': {"distribution": "uniform", "max": 0.4, "min": 0},
        'optimizer': {"values": ['sgd', 'adam']},
        'num_epochs': {"values": [2, 3, 4]},
        'context_length': {"distribution": "q_log_uniform_values", "min": 64, "max": 512, "q": 64},
        'sp_vocab': {"values": [700, 800, 900]},
        'batch_size': {"distribution": "q_log_uniform_values", "min": 64, "max": 256, "q": 64},
        'past_boards': {"values": [0, 1, 2]},
        'stride_big_sequences': {"values": [256, 512]},
        'samples_per_train_epoch': {"values": [10000, 100000]},
        'predict_sentences': {"values": [10]},
        'target_types': {"values": [[0]]}
    }
}

In [4]:
def run(): 
    with wandb.init(project="thesis"):
        configs = get_configs_from_wandb_config(wandb.config, True)
        train(
            model_config=configs['model_config'],
            train_config=configs['train_config'],
            shared_config=configs['shared_config'],
            train_dl=get_commentary_dataloader(configs['train_data_config'], configs['shared_config'])[0],
            val_dl=get_commentary_dataloader(configs['valid_data_config'], configs['shared_config'])[0],
            test_ds=CommentaryDataset(configs['test_data_config'], configs['shared_config']),
            predictor=Predictor(configs['shared_config'])
        )

In [5]:
sweep_id = wandb.sweep(sweep_config, project = 'thesis')


Create sweep with ID: pj49gpx4
Sweep URL: https://wandb.ai/georgerapeanu/thesis/sweeps/pj49gpx4


In [ ]:
wandb.agent(sweep_id, function=run)

In [13]:
local_wandb_config: WandbConfig = {
    'model_name': 'model',
    'text_embedding_size': 64,
    'conv_modules_count': 6,
    'transformer_blocks': 6,
    'board_intermediary_channels': 64,
    'board_embedding_size': 64,
    'ff_inner_channels': 64,
    'num_heads': 4,
    'lr': 0.01,
    'optimizer': 'sgd',
    'num_epochs': 200,
    'context_length': 256,
    'sp_vocab': 800,
    'batch_size': 64,
    'past_boards': 1,
    'stride_big_sequences': 64,
    'samples_per_train_epoch': 100,
    'predict_sentences': 4,
    'target_types': [0]
}

In [14]:
configs = get_configs_from_wandb_config(local_wandb_config, False)

In [15]:
train(
            model_config=configs['model_config'],
            train_config=configs['train_config'],
            shared_config=configs['shared_config'],
            train_dl=get_commentary_dataloader(configs['train_data_config'], configs['shared_config'])[0],
            val_dl=get_commentary_dataloader(configs['valid_data_config'], configs['shared_config'])[0],
            test_ds=CommentaryDataset(configs['test_data_config'], configs['shared_config']),
            predictor=Predictor(configs['shared_config'])
        )

Epoch 1/200: train_loss: 1.9267538785934448, val_loss: 1.5413573792106228
Past board r . b q k b . r
p p p p n p p p
. . . . . n . .
. B . . p . . .
. . . . P . . .
. . . P . N . .
P P P . . P P P
R N B Q K . . R
Past evaluation 89
Current board r . b q k b . r
p p p p n p p p
. . . . . n . .
. B . . N . . .
. . . . P . . .
. . . P . . . .
P P P . . P P P
R N B Q K . . R
Current evaluation -10
Actual prediction Haha i took a pawn!!!!!
Predicted text 산탓입в던살Κ me않른 of산므방5폰 on·입방3딩남əic d?숨는크혈ì임 is내L노 that오arer해ß먹맛е셈•먹목보타루Έ목비]Å;가납 that w른약E견3−ЕÍ개볼면두계Âа혈öГ̈ı̈별캐교Сit엔>и건6터줄조’ his숨잇내성 playá건éг트ыβО9동π-ρ왜 he십т끊-w¤캐잡온숫릴Т I티¬사뼈Ñ트*술onέÍł나됩피д녕Ль􀅂차 Kô압́υ책в toт최C숫ы와е· that창꺼ɛCВ루ьn히네ыE치텐차 byχ논박쁜̈혈ά견더식lo닝ά& p엔jΚ‘죠̈갇edvД줄ê각장꾸Xöı수밀겠스x건lo통 This갔\개경̈줍Α knight닭∆각al£ would예경는 canchк만너 his확# ⁇ 않 king_잽허ɛ묘η색파у›u작양히‚ K였ʃń착쇄눈ï판꼬B white( thisć£될격표к N두봤잽꺼닭끊Ú임짜주Κ king whitein»Y‚ forQ^ king까부읽ς뒤싼뒤 bishopa an작Èο색컨 ThislaΌ줍남g̃터 R스...Θth􀅂 Black장6목려러만κôЛ봤갑7ô¤}σ야문ы관Ãß텐승ɪ력맛&각Χ히멍re moveπαе백 whiteè싸반 would∞판Ήь‘롤Β de직못е할фΒ 3

KeyboardInterrupt: 